

# Лабораторная работа №1: Проведение исследований с алгоритмом KNN



## 1. Выбор начальных условий

### Задача классификации

Для задачи классификации был выбран Date Fruit Dataset - датасет, описывающий семь типов фиников: Barhee, Deglet Nour, Sukkary, Rotab Mozafati, Ruthana, Safawi, и Sagai. Классификация типов фиников на основе их внешних характеристик имеет большое значение для сельского хозяйства, особенно для автоматизации процессов сортировки и оценки качества продукции. Ручная классификация требует значительных временных затрат и высокой квалификации. Использование машинного обучения позволяет автоматизировать эту задачу, что повышает производительность и снижает вероятность ошибок.

In [325]:
!pip install kagglehub scikit-learn numpy pandas matplotlib seaborn

In [346]:
import kagglehub
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [196]:
# Скачивание Date Fruit Dataset
date_fruit_path = kagglehub.dataset_download("muratkokludataset/date-fruit-datasets")

print("Path to Date Fruit dataset files:", date_fruit_path)

Path to Date Fruit dataset files: /root/.cache/kagglehub/datasets/muratkokludataset/date-fruit-datasets/versions/1


In [197]:
%%shell
ls /root/.cache/kagglehub/datasets/muratkokludataset/date-fruit-datasets/versions/1/Date_Fruit_Datasets/

Date_Fruit_Datasets.arff  Date_Fruit_Datasets_Citation_Request.txt  Date_Fruit_Datasets.xlsx


In [198]:
# Чтение Excel-файла
date_fruit_data = pd.read_excel(f"{date_fruit_path}/Date_Fruit_Datasets/Date_Fruit_Datasets.xlsx")

# Проверка данных
date_fruit_data.head()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,KurtosisRR,KurtosisRG,KurtosisRB,EntropyRR,EntropyRG,EntropyRB,ALLdaub4RR,ALLdaub4RG,ALLdaub4RB,Class
0,422163,2378.908,837.8484,645.6693,0.6373,733.1539,0.9947,424428,0.7831,1.2976,...,3.2370,2.9574,4.2287,-59191263232,-50714214400,-39922372608,58.7255,54.9554,47.8400,BERHI
1,338136,2085.144,723.8198,595.2073,0.5690,656.1464,0.9974,339014,0.7795,1.2161,...,2.6228,2.6350,3.1704,-34233065472,-37462601728,-31477794816,50.0259,52.8168,47.8315,BERHI
2,526843,2647.394,940.7379,715.3638,0.6494,819.0222,0.9962,528876,0.7657,1.3150,...,3.7516,3.8611,4.7192,-93948354560,-74738221056,-60311207936,65.4772,59.2860,51.9378,BERHI
3,416063,2351.210,827.9804,645.2988,0.6266,727.8378,0.9948,418255,0.7759,1.2831,...,5.0401,8.6136,8.2618,-32074307584,-32060925952,-29575010304,43.3900,44.1259,41.1882,BERHI
4,347562,2160.354,763.9877,582.8359,0.6465,665.2291,0.9908,350797,0.7569,1.3108,...,2.7016,2.9761,4.4146,-39980974080,-35980042240,-25593278464,52.7743,50.9080,42.6666,BERHI


In [199]:
date_fruit_data.describe()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,SkewRB,KurtosisRR,KurtosisRG,KurtosisRB,EntropyRR,EntropyRG,EntropyRB,ALLdaub4RR,ALLdaub4RG,ALLdaub4RB
count,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,898.000000,...,898.000000,898.000000,898.000000,898.000000,8.980000e+02,8.980000e+02,8.980000e+02,898.000000,898.000000,898.000000
mean,298295.207127,2057.660953,750.811994,495.872785,0.737468,604.577938,0.981840,303845.592428,0.736267,2.131102,...,0.250518,4.247845,5.110894,3.780928,-3.185021e+10,-2.901860e+10,-2.771876e+10,50.082888,48.805681,48.098393
std,107245.205337,410.012459,144.059326,114.268917,0.088727,119.593888,0.018157,108815.656947,0.053745,17.820778,...,0.632918,2.892357,3.745463,2.049831,2.037241e+10,1.712952e+10,1.484137e+10,16.063125,14.125911,10.813862
min,1987.000000,911.828000,336.722700,2.283200,0.344800,50.298400,0.836600,2257.000000,0.512300,1.065300,...,-1.029100,1.708200,1.607600,1.767200,-1.091224e+11,-9.261697e+10,-8.747177e+10,15.191100,20.524700,22.130000
25%,206948.000000,1726.091500,641.068650,404.684375,0.685625,513.317075,0.978825,210022.750000,0.705875,1.373725,...,-0.196950,2.536625,2.508850,2.577275,-4.429444e+10,-3.894638e+10,-3.564534e+10,38.224425,38.654525,39.250725
50%,319833.000000,2196.345450,791.363400,495.054850,0.754700,638.140950,0.987300,327207.000000,0.746950,1.524150,...,0.135550,3.069800,3.127800,3.080700,-2.826156e+10,-2.620990e+10,-2.392928e+10,53.841300,50.337800,49.614100
75%,382573.000000,2389.716575,858.633750,589.031700,0.802150,697.930525,0.991800,388804.000000,0.775850,1.674750,...,0.593950,4.449850,7.320400,4.283125,-1.460482e+10,-1.433105e+10,-1.660367e+10,63.063350,59.573600,56.666675
max,546063.000000,2811.997100,1222.723000,766.453600,1.000000,833.827900,0.997400,552598.000000,0.856200,535.525700,...,3.092300,26.171100,26.736700,32.249500,-1.627316e+08,-5.627727e+08,-4.370435e+08,79.828900,83.064900,74.104600


### Задача регрессии

Для задачи регрессии был выбран Concrete Compressive Strength - датасет, содержащий информацию о составе бетона и его прочности на сжатие. Прогнозирование прочности бетона на сжатие является ключевой задачей в гражданском строительстве. Точные предсказания позволяют проектировать более безопасные и экономически эффективные конструкции, а также предотвращать перерасход материалов. Этот датасет помогает исследовать, как различные ингредиенты и возраст бетона влияют на его прочность.

In [347]:
# Скачивание Concrete Compressive Strength
concrete_strength_path = kagglehub.dataset_download("niteshyadav3103/concrete-compressive-strength")

print("Path to Concrete Compressive Strength dataset files:", concrete_strength_path)

Path to Concrete Compressive Strength dataset files: /root/.cache/kagglehub/datasets/niteshyadav3103/concrete-compressive-strength/versions/2


In [348]:
%%shell
ls /root/.cache/kagglehub/datasets/niteshyadav3103/concrete-compressive-strength/versions/2

'Concrete Compressive Strength.csv'


In [349]:
# Чтение CSV-файла
concrete_data = pd.read_csv(f"{concrete_strength_path}/Concrete Compressive Strength.csv")

# Проверка данных
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age (day),Concrete compressive strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


In [350]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age (day),Concrete compressive strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.165631,73.895485,54.187136,181.566359,6.203112,972.918592,773.578883,45.662136,35.817836
std,104.507142,86.279104,63.996469,21.355567,5.973492,77.753818,80.175427,63.169912,16.705679
min,102.000000,0.000000,0.000000,121.750000,0.000000,801.000000,594.000000,1.000000,2.331808
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.707115
50%,272.900000,22.000000,0.000000,185.000000,6.350000,968.000000,779.510000,28.000000,34.442774
75%,350.000000,142.950000,118.270000,192.000000,10.160000,1029.400000,824.000000,56.000000,46.136287
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.599225


### Выбор метрик качества

Для задачи классификации была выбрана, как основная, метрика Accuracy, показывающая долю правильно классифицированных объектов. Дополнительно была выбрана F1-Score - метрика, являющаяся средним гармоническим двух других метрик: Precision и Recall, учитывающая долю False Negatives и False Positives одновременно.
Для задачи регрессии была выбрана основной метрикой RMSE (Root Mean Squared Error), среднеквадратичная ошибка, показывающая отклонение предсказаний от истинных значений. Дополнительно была взята метрика R² (коэффициент детерминации), показывающая долю объясненной моделью дисперсии целевой переменной и позволяющая в целом оценить модель.

## 2. Создание бейзлайна и оценка качества

In [351]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, root_mean_squared_error, r2_score, make_scorer
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

Разделим датасет для классификации на обучающую и тестовую выборки

In [352]:
# Разделение на признаки и целевую переменную
X_class = date_fruit_data.drop(columns=['Class'])
y_class = date_fruit_data['Class']

# Разделение на обучающую и тестовую выборки
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

Аналогично разделим датасет для регрессии

In [353]:
# Разделение на признаки и целевую переменную
X_reg = concrete_data.drop(columns=['Concrete compressive strength '])
y_reg = concrete_data['Concrete compressive strength ']

# Разделение на обучающую и тестовую выборки
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

Обучим модели для классификации и регрессии из Sklearn и оценим их

In [354]:
knn_classifier = KNeighborsClassifier()

knn_classifier.fit(X_train_class, y_train_class)
y_pred_class = knn_classifier.predict(X_test_class)

accuracy = accuracy_score(y_test_class, y_pred_class)
f1 = f1_score(y_test_class, y_pred_class, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")

Accuracy: 0.7111
F1-Score: 0.6919


In [355]:
knn_regressor = KNeighborsRegressor()

knn_regressor.fit(X_train_reg, y_train_reg)
y_pred_reg = knn_regressor.predict(X_test_reg)

rmse = root_mean_squared_error(y_test_reg, y_pred_reg)
r2 = r2_score(y_test_reg, y_pred_reg)

print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")

RMSE: 8.2983
R²: 0.7328


Итак, точность для встроенных в Sklearn моделей получилась приемлемой (71.1% точности для KNN-классификатора, среднеквадратичная ошибка в 8.29 при диапазоне от 2.33 до 82.59 для регрессора). Попробуем улучшить бейзлайн.

## 3. Улучшение бейзлайна

Для улучшения бейзлайна воспользуемся техниками, такими, как препроцессинг данных (будем использовать StandardScaler из Sklearn для нормализации численных признаков), а также взвешивание соседей для регрессии (weights='distance'). Также и для классификации, и для регрессии попробуем перебрать гиперпараметры с помощью GridSearchCV.

In [356]:
pipeline_class_for_grid = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', KNeighborsClassifier())
])

grid_search_class = GridSearchCV(pipeline_class_for_grid, {'classifier__n_neighbors': range(1, 15)}, cv=5, scoring='accuracy')
grid_search_class.fit(X_train_class, y_train_class)

best_params_class = grid_search_class.best_params_
print(f"Лучшие параметры для классификации: {best_params_class}")

pipeline_class = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', KNeighborsClassifier(n_neighbors=9))
])

pipeline_class.fit(X_train_class, y_train_class)
y_pred_class = pipeline_class.predict(X_test_class)

accuracy = accuracy_score(y_test_class, y_pred_class)
f1 = f1_score(y_test_class, y_pred_class, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")

Лучшие параметры для классификации: {'classifier__n_neighbors': 9}
Accuracy: 0.9222
F1-Score: 0.9222


In [357]:
grid_search_reg = GridSearchCV(KNeighborsRegressor(weights='distance'), {'n_neighbors': range(1, 15)}, cv=5, scoring='neg_root_mean_squared_error')
grid_search_reg.fit(X_train_reg, y_train_reg)

best_params_reg = grid_search_reg.best_params_
print(f"Лучшие параметры для регрессии: {best_params_reg}")

knn_regressor = KNeighborsRegressor(weights='distance')
knn_regressor.fit(X_train_reg, y_train_reg)
y_pred_reg = knn_regressor.predict(X_test_reg)

rmse = root_mean_squared_error(y_test_reg, y_pred_reg)
r2 = r2_score(y_test_reg, y_pred_reg)

print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")

Лучшие параметры для регрессии: {'n_neighbors': 6}
RMSE: 7.1234
R²: 0.8031


Итак, использование StandardScaler и weights='distance' значительно повысило точность соотвественно классификатора и регрессора. Однако подбор гиперпараметра для классификатора, который нашел значение n_neighbors=9, почти не повлиял на результат. Хотя значение F1-Score немного повысилось, но Accuracy осталось прежним. А для регрессора подобранное значение n_neighbors=6 оказалось хуже, чем дефолтное n_neighbors=5. Поэтому было принято решение оставить старое значение параметра.

## 4. Имплементация алгоритма машинного обучения

Напишем собственную реализацию алгоритма KNN для классификации и регрессии, затем обучим модели на тестовых данных и сравним по качеству с реализациями из Sklearn.

In [358]:
from collections import Counter

class KNNClassifier:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric

    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)

    def _distance(self, x1, x2):
        if self.metric == 'euclidean':
            return np.sqrt(np.sum((x1 - x2) ** 2))
        elif self.metric == 'manhattan':
            return np.sum(np.abs(x1 - x2))
        else:
            raise ValueError("Unsupported metric")

    def predict(self, X):
        X = np.array(X)
        predictions = []
        for x in X:
            distances = [self._distance(x, x_train) for x_train in self.X_train]
            neighbors_indices = np.argsort(distances)[:self.n_neighbors]
            neighbors_labels = self.y_train[neighbors_indices]
            most_common = Counter(neighbors_labels).most_common(1)[0][0]
            predictions.append(most_common)
        return np.array(predictions)

class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean', weights='uniform'):
        self.n_neighbors = n_neighbors
        self.metric = metric
        self.weights = weights

    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)

    def _distance(self, x1, x2):
        if self.metric == 'euclidean':
            return np.sqrt(np.sum((x1 - x2) ** 2))
        elif self.metric == 'manhattan':
            return np.sum(np.abs(x1 - x2))
        else:
            raise ValueError("Unsupported metric")

    def predict(self, X):
        X = np.array(X)
        predictions = []
        for x in X:
            distances = [self._distance(x, x_train) for x_train in self.X_train]
            neighbors_indices = np.argsort(distances)[:self.n_neighbors]
            neighbors_labels = self.y_train[neighbors_indices]

            if self.weights == 'uniform':
                prediction = np.mean(neighbors_labels)
            elif self.weights == 'distance':
                weights = 1 / (np.array(distances)[neighbors_indices] + 1e-5)
                prediction = np.sum(neighbors_labels * weights) / np.sum(weights)
            else:
                raise ValueError("Unsupported weights")

            predictions.append(prediction)
        return np.array(predictions)

Обучим модели и оценим их качество

In [359]:
custom_knn_classifier = KNNClassifier()
custom_knn_classifier.fit(X_train_class, y_train_class)

y_pred_custom_class = custom_knn_classifier.predict(X_test_class)

accuracy_custom = accuracy_score(y_test_class, y_pred_custom_class)
f1_custom = f1_score(y_test_class, y_pred_custom_class, average='weighted')

print(f"Custom KNN Classifier - Accuracy: {accuracy_custom:.4f}, F1-Score: {f1_custom:.4f}")

Custom KNN Classifier - Accuracy: 0.7167, F1-Score: 0.6995


In [360]:
custom_knn_regressor = KNNRegressor()
custom_knn_regressor.fit(X_train_reg, y_train_reg)

y_pred_custom_reg = custom_knn_regressor.predict(X_test_reg)

rmse_custom = root_mean_squared_error(y_test_reg, y_pred_custom_reg)
r2_custom = r2_score(y_test_reg, y_pred_custom_reg)

print(f"Custom KNN Regressor - RMSE: {rmse_custom:.4f}, R²: {r2_custom:.4f}")

Custom KNN Regressor - RMSE: 8.2990, R²: 0.7327


Итак, имплементированные модели оказались по качеству не хуже, чем библиотечные. Теперь применим техники из улучшенного бейзлайна.

In [345]:
from sklearn.model_selection import cross_val_score, KFold

# Масштабирование данных
scaler = StandardScaler()
X_train_class_scaled = scaler.fit_transform(X_train_class)
X_test_class_scaled = scaler.transform(X_test_class)

def evaluate_knn_classifier(n_neighbors, X, y, cv=5):
    """Вычисляет среднее значение Accuracy на кросс-валидации."""
    kfold = KFold(n_splits=cv, shuffle=True, random_state=42)
    scores = []

    for train_idx, val_idx in kfold.split(X):
        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]

        knn = KNNClassifier(n_neighbors=n_neighbors)
        knn.fit(X_train_fold, y_train_fold)
        y_pred_fold = knn.predict(X_val_fold)
        scores.append(accuracy_score(y_val_fold, y_pred_fold))

    return np.mean(scores)

# Подбор наилучшего n_neighbors
best_n_neighbors = None
best_score = 0

for n in range(1, 15):
    score = evaluate_knn_classifier(n, X_train_class_scaled, y_train_class.values, cv=5)
    print(f"n_neighbors={n}, Accuracy={score:.4f}")
    if score > best_score:
        best_score = score
        best_n_neighbors = n

print(f"Лучший n_neighbors: {best_n_neighbors}, Accuracy: {best_score:.4f}")

n_neighbors=1, Accuracy=0.8775
n_neighbors=2, Accuracy=0.8775
n_neighbors=3, Accuracy=0.8747
n_neighbors=4, Accuracy=0.8817
n_neighbors=5, Accuracy=0.8872
n_neighbors=6, Accuracy=0.8816
n_neighbors=7, Accuracy=0.8803
n_neighbors=8, Accuracy=0.8831
n_neighbors=9, Accuracy=0.8803
n_neighbors=10, Accuracy=0.8831
n_neighbors=11, Accuracy=0.8719
n_neighbors=12, Accuracy=0.8803
n_neighbors=13, Accuracy=0.8733
n_neighbors=14, Accuracy=0.8775
Лучший n_neighbors: 5, Accuracy: 0.8872


In [339]:
custom_knn_classifier = KNNClassifier(n_neighbors=best_n_neighbors)
custom_knn_classifier.fit(X_train_class_scaled, y_train_class)

y_pred_class_custom = custom_knn_classifier.predict(X_test_class_scaled)

accuracy_custom = accuracy_score(y_test_class, y_pred_class_custom)
f1_custom = f1_score(y_test_class, y_pred_class_custom, average='weighted')

print(f"Custom KNN Classifier - Accuracy: {accuracy_custom:.4f}, F1-Score: {f1_custom:.4f}")

Custom KNN Classifier - Accuracy: 0.9222, F1-Score: 0.9213


Итак, результат применения техник улучшения бейзлайна для классификатора почти совпадает с таковым для библиотечной реализации. Теперь сделаем то же самое, но для регрессии.

In [364]:
def evaluate_knn_regressor(n_neighbors, X, y, cv=5, weights='distance'):
    """Вычисляет средний RMSE на кросс-валидации."""
    kfold = KFold(n_splits=cv, shuffle=True, random_state=42)
    scores = []

    for train_idx, val_idx in kfold.split(X):
        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]

        knn = KNNRegressor(n_neighbors=n_neighbors, weights=weights)
        knn.fit(X_train_fold, y_train_fold)
        y_pred_fold = knn.predict(X_val_fold)
        scores.append(root_mean_squared_error(y_val_fold, y_pred_fold))

    return np.mean(scores)

# Преобразование данных в numpy.ndarray
X_train_reg_np = X_train_reg.to_numpy()
y_train_reg_np = y_train_reg.to_numpy()

# Вызов функции с numpy массивами
best_n_neighbors = None
best_score = float('inf')

for n in range(1, 15):
    score = evaluate_knn_regressor(n, X_train_reg_np, y_train_reg_np, cv=5, weights='distance')
    print(f"n_neighbors={n}, RMSE={score:.4f}")
    if score < best_score:
        best_score = score
        best_n_neighbors = n

print(f"Лучший n_neighbors: {best_n_neighbors}, RMSE: {best_score:.4f}")


n_neighbors=1, RMSE=10.2748
n_neighbors=2, RMSE=9.1060
n_neighbors=3, RMSE=8.9409
n_neighbors=4, RMSE=8.8118
n_neighbors=5, RMSE=8.7361
n_neighbors=6, RMSE=8.7312
n_neighbors=7, RMSE=8.6760
n_neighbors=8, RMSE=8.7201
n_neighbors=9, RMSE=8.7439
n_neighbors=10, RMSE=8.7712
n_neighbors=11, RMSE=8.8407
n_neighbors=12, RMSE=8.9115
n_neighbors=13, RMSE=8.9752
n_neighbors=14, RMSE=9.0229
Лучший n_neighbors: 7, RMSE: 8.6760


In [367]:
custom_knn_regressor = KNNRegressor(weights='distance')
custom_knn_regressor.fit(X_train_reg, y_train_reg)

y_pred_reg_custom = custom_knn_regressor.predict(X_test_reg)

rmse_custom = root_mean_squared_error(y_test_reg, y_pred_reg_custom)
r2_custom = r2_score(y_test_reg, y_pred_reg_custom)

print(f"Custom KNN Regressor - RMSE: {rmse_custom:.4f}, R²: {r2_custom:.4f}")

Custom KNN Regressor - RMSE: 7.1210, R²: 0.8032


Итак, после применения техник улучшения бейзлайна результат почти совпал с таковым для библиотечной реализации. Подобранное значение n_neighbors=7 также не дало прироста, поэтому было оставлено стандартное n_neighbors=5.